In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from pygcn.utils import load_data, accuracy
from pygcn.models import GCN
from pygcn.train import train, test

In [3]:
# Setting
seed = 42
num_epoch = 201
lr = 0.01
weight_decay = 5e-4
hidden = 16
dropout = 0.5
cuda = False

In [4]:
np.random.seed(seed)
torch.manual_seed(seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data(path="data/cora/")

# Model and optimizer
model = GCN(nfeat=features.shape[1],
                nhid=hidden,
                nclass=labels.max().item() + 1,
                dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                        lr=lr, weight_decay=weight_decay)


Loading cora dataset...
Done loading cora dataset


In [5]:
# Move to cuda if use GPU
if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [6]:
# Train model
train(features, adj, labels, idx_train, idx_val, model, optimizer, num_epoch)    

Epoch: 0001 loss_train: 1.9775 acc_train: 0.1286 loss_val: 1.9681 acc_val: 0.1567 time: 0.9265s
Epoch: 0021 loss_train: 1.7478 acc_train: 0.2929 loss_val: 1.7282 acc_val: 0.3500 time: 0.0220s
Epoch: 0041 loss_train: 1.5040 acc_train: 0.4000 loss_val: 1.5538 acc_val: 0.4000 time: 0.0250s
Epoch: 0061 loss_train: 1.1857 acc_train: 0.6857 loss_val: 1.3011 acc_val: 0.6300 time: 0.0280s
Epoch: 0081 loss_train: 0.9250 acc_train: 0.7929 loss_val: 1.0720 acc_val: 0.7733 time: 0.0240s
Epoch: 0101 loss_train: 0.6933 acc_train: 0.9071 loss_val: 0.9171 acc_val: 0.8067 time: 0.0240s
Epoch: 0121 loss_train: 0.6232 acc_train: 0.9071 loss_val: 0.8253 acc_val: 0.8000 time: 0.0230s
Epoch: 0141 loss_train: 0.5003 acc_train: 0.9286 loss_val: 0.7732 acc_val: 0.8033 time: 0.0240s
Epoch: 0161 loss_train: 0.4544 acc_train: 0.9357 loss_val: 0.7305 acc_val: 0.8100 time: 0.0230s
Epoch: 0181 loss_train: 0.4096 acc_train: 0.9643 loss_val: 0.6992 acc_val: 0.8167 time: 0.0260s
Epoch: 0201 loss_train: 0.3952 acc_train

In [7]:
# Test model
test(features, adj, labels, idx_test, model)

Test set results: loss= 0.7144 accuracy= 0.8320


In [8]:
# shape of final GCN output
# 2708 nodes, 7 features each
model.eval()
output = model(features, adj)
output_2 = output.cpu().detach().numpy()
#save embeddings in numpy format file
np.save('embedding_2.npy', output_2)
print(output.shape)

torch.Size([2708, 7])


In [9]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [10]:
import pandas as pd

In [11]:
feat_cols = [ 'dim'+str(i) for i in range(output.shape[1])]
df = pd.DataFrame(output.detach().cpu().numpy(), columns=feat_cols)
df['label'] = labels
print('Size of the dataframe: {}'.format(df.shape))

Size of the dataframe: (2708, 8)


In [12]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(df[feat_cols].values)
df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.49270335 0.1948728  0.11997908]


In [13]:
print(df[feat_cols].values)

[[-4.956924   -3.9997063  -5.618423   ... -5.953792   -4.9827
  -5.5522656 ]
 [-1.6026056  -3.686058   -0.8451312  ... -3.2764738  -2.192847
  -1.8243399 ]
 [-3.9293327  -4.8059464  -3.864922   ... -4.265363   -0.13311777
  -3.3009799 ]
 ...
 [-2.4777071  -3.6391432  -2.682134   ... -0.4411228  -2.7484808
  -2.753981  ]
 [-3.916076   -3.286599   -2.6711898  ... -3.4282498  -2.4897256
  -0.32284778]
 [-4.4061756  -3.826961   -4.455226   ... -4.430542   -4.384447
  -4.169762  ]]


In [14]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="label",
    palette=sns.color_palette("hls", 7),
    data=df.loc[:,:],
    legend="full",
    alpha=0.3
)

AttributeError: module 'seaborn' has no attribute 'scatterplot'

<Figure size 1152x720 with 0 Axes>

In [ ]:
ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=df.loc[:,:]["pca-one"], 
    ys=df.loc[:,:]["pca-two"], 
    zs=df.loc[:,:]["pca-three"], 
    c=df.loc[:,:]["label"], 
    cmap='tab10'
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=10000)
tsne_results = tsne.fit_transform(df[feat_cols].values)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="label",
    palette=sns.color_palette("hls", 7),
    data=df,
    legend="full",
    alpha=0.3
)

In [ ]:
# After you train the model, you can try to export the embeddings into the file (numpy)
# You can train and generate the embeddings with another seed
# You can try to calculate the alignment matrix by using the dot product
# You can try to visualize the two embedding spaces